In [1]:
import pandas as pd
import json

In [13]:
!pip install vllm

     ---------------------------------------- 0.0/874.1 kB ? eta -:--:--
     -- ---------------------------------- 61.4/874.1 kB 825.8 kB/s eta 0:00:01
     ------- ------------------------------ 174.1/874.1 kB 1.8 MB/s eta 0:00:01
     ------------- ------------------------ 307.2/874.1 kB 2.1 MB/s eta 0:00:01
     ------------------- ------------------ 450.6/874.1 kB 2.4 MB/s eta 0:00:01
     ------------------------------------ - 829.4/874.1 kB 3.5 MB/s eta 0:00:01
     -------------------------------------- 874.1/874.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\andre\AppData\Local\Temp\pip-build-env-o7pm84rg\

In [2]:
with open('datasets/hellaswag_ro/ro_validation.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)


In [3]:
df

,ctx_a,ctx_b,ctx,endings,id,ind,activity_label,source_id,split,split_type,label
0,[header] Cum să obții o coafură drăguță pentru...,,[header] Cum să obții o coafură drăguță pentru...,"[[title] Următorul pas, descurcă ușor nodurile...",hellaswag/validation/4478,9492,Personal Care and Style,wikihow~15665,val,zeroshot,0
1,O femeie într-un costum roșu vorbește în spate...,un bărbat,O femeie într-un costum roșu vorbește în spate...,"[ia mingile de pe un suport de pe perete., fol...",hellaswag/validation/1559,23536,Running a marathon,activitynet~v_eyfUkLbsixg,val,zeroshot,3
2,[header] Cum să faci un scrub cu zahăr și vani...,,[header] Cum să faci un scrub cu zahăr și vani...,[[title] Toarnă 3 lingurițe de vanilie natural...,hellaswag/validation/7065,28378,Personal Care and Style,wikihow~46745,val,zeroshot,0
3,[header] Cum să porți pantofi roz [title] Aleg...,,[header] Cum să porți pantofi roz [title] Aleg...,"[Dacă doriți ceva sclipitor, alegeți pantofi r...",hellaswag/validation/9492,46170,Personal Care and Style,wikihow~75990,val,zeroshot,2
4,[header] Cum să îndepărtezi mirosul de câine d...,,[header] Cum să îndepărtezi mirosul de câine d...,[Dacă observi că mașina ta miroase precum tine...,hellaswag/validation/7882,34272,Education and Communications,wikihow~56427,val,indomain,1
...,...,...,...,...,...,...,...,...,...,...,...
9240,[header] Cum să scapi de zona întunecată din j...,,[header] Cum să scapi de zona întunecată din j...,[Acestea pot include următoarele: [substeps] E...,hellaswag/validation/6957,27533,Health,wikihow~45375,val,indomain,2
9241,"Bărbatul apoi o ridică, face genuflexiuni, sar...",bărbatul,"Bărbatul apoi o ridică, face genuflexiuni, sar...",[apoi își rotunjește picioarele și câștigă imp...,hellaswag/validation/571,7924,Clean and jerk,activitynet~v_3bi2XM3scQA,val,zeroshot,3
9242,[header] Cum să blanșezi morcovii [title] Spal...,,[header] Cum să blanșezi morcovii [title] Spal...,[[step] Folosește un căluț de legume pentru a ...,hellaswag/validation/6172,21757,Food and Entertaining,wikihow~35924,val,indomain,0
9243,Un bărbat este arătat lucrând într-o grădină d...,el,Un bărbat este arătat lucrând într-o grădină d...,"[folosește unelte în timp ce lucrează., folose...",hellaswag/validation/805,11165,Trimming branches or hedges,activitynet~v_D5hsFA2ZrIQ,val,indomain,0


In [4]:
llama3_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.<|eot_id|><|start_header_id|>user<|end_header_id|>

A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

llama2_prompt = """<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.
<</SYS>>

A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d} [/INST]
"""

alpaca_prompt = """Mai jos veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.

### Instrucțiune:
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}

### Răspuns:
"""

In [8]:
import requests

def generate_response(prompt):
    url = "http://localhost:8000/generate"

    payload = {
        "prompt": prompt,
        "n": 1,
        "temperature": 0.95,
        "max_tokens": 1024
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

def get_chosen_answer(row):
    response_data = generate_response(row['formatted_prompt'])
    if response_data:
        answer = response_data['text'][0].split('\n')[-1]
        with open("hellaswag_responses_llama3_ro", 'a') as f:
            f.write(f"Question: {row['formatted_prompt']}\nAnswer: {answer}\n\n")
        return answer
    else:
        return None


In [6]:
def format_prompt(row, model='llama3'):
    # Combine the context with endings
    context = row['ctx']
    options = row['endings']
    
    # Use the correct prompt template
    if model == 'llama3':
        prompt_template = llama3_prompt
    elif model == 'llama2':
        prompt_template = llama2_prompt
    elif model == 'alpaca':
        prompt_template = alpaca_prompt
    else:
        raise ValueError("Unknown model specified")

    # Format the prompt
    prompt = prompt_template.format(
        context=context,
        option_a=options[0],
        option_b=options[1],
        option_c=options[2],
        option_d=options[3]
    )
    
    return prompt

In [7]:
df['formatted_prompt'] = df.apply(lambda row: format_prompt(row, model='llama3'), axis=1)

df['formatted_prompt'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSunteți un asistent util, respectuos și onest. Veți primi 4 variante de propoziții având același context de început, dar continuări diferite. O singură variantă este corectă. Răspundeți doar cu varianta care completează contextul în mod corespunzător.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nA. [title] Următorul pas, descurcă ușor nodurile, începând de la vârfuri și lucrând încet spre rădăcini. [title] Împarte-ți părul în două părți - jumătatea de sus și jumătatea de jos.\nB. [title] Fă-ți părul pufos cu un pieptene cu dinți mari dacă este des sau creț. [title] Menține-ți părul uscat înfășurând o bandă elastică în jurul unui pieptene, la fel cum ai face cu părul tău.\nC. [title] Iată un șampon uscat și un balsam. [step] Are o cană și ar trebui să fie prima dată în părul tău.\nD. [title] Poți purta o coafură făcută pentru o zi obișnuită. [step] Poți să o porți cu o torsadă sau să o lași jos.<|eot_id|><|start_hea

In [ ]:
formatted_prompt = df['formatted_prompt'][1]

formatted_prompt

In [ ]:
df['response'] = df.apply(lambda row: get_chosen_answer(row), axis=1)


In [ ]:
df

In [ ]:
new_df = df['question',	'mc1_targets_choices', 'mc2_targets_choices',	'mc1_targets_labels',	'mc2_targets_labels', 'response'].copy()
# new_df.rename(columns={'answer': 'correct_answer'}, inplace=True)

In [ ]:
df.to_csv('hellaswag_llama3_ro.csv', index=False, encoding='utf-8')

In [11]:
from sentence_transformers import SentenceTransformer, util

# Load the multilingual sentence transformer
model_name = 'sentence-transformers/paraphrase-xlm-r-multilingual-v1'
model = SentenceTransformer(model_name)

# Example sentences
sentence1 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China cheltuiește cel mai mult pe turismul internațional."
sentence2 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China."

# Compute embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
cosine_sim = util.cos_sim(embedding1, embedding2)
print(f"Cosine Similarity: {cosine_sim.item():.4f}")


Cosine Similarity: 0.9833


In [12]:
from sentence_transformers import SentenceTransformer, util

# Load the multilingual sentence transformer
model_name = 'BlackKakapo/stsb-xlm-r-multilingual-ro'
model = SentenceTransformer(model_name)

# Example sentences
sentence1 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China cheltuiește cel mai mult pe turismul internațional."
sentence2 = "Care țară mare cheltuiește cel mai mult pe turismul internațional? China."

# Compute embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
cosine_sim = util.cos_sim(embedding1, embedding2)
print(f"Cosine Similarity: {cosine_sim.item():.4f}")


Cosine Similarity: 0.9831
